In [12]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import time
# import vectorbt as vbt
pd.options.mode.chained_assignment = None

In [13]:

df = pd.read_csv('15sTQQQ.csv')
df = df.dropna()
df

df['Time'] = pd.DatetimeIndex(pd.to_datetime(df['Time'].str[:19]))
df = df.set_index('Time')

df['tr'] = np.maximum.reduce([
    abs(df['High'] - df['Low']),
    abs(df['High'] - df['Close'].shift()),
    abs(df['Low'] - df['Close'].shift())
])
df['atr'] = ta.atr(df.High,df.Low,df.Close,14)

df = df.between_time("9:30", "15:58")

In [14]:
def V(closes,opens,tr,atr,startCount,countCandleV):
    v = False
    if countCandleV >= 3 and countCandleV <= 8 and closes > opens:
        v = True
    if -tr < -atr:
        startCount = 1
    if closes < opens and startCount == 1: 
        countCandleV = countCandleV + 1
    else:
        countCandleV = 0
        startCount = 0

    return(v,countCandleV,startCount)

In [15]:
df['entries'] = np.nan
df

,Unnamed: 0,Open,Close,High,Low,Volume,tr,atr,entries
Time,,,,,,,,,
2022-02-01 09:30:00,985,62.4200,62.3500,62.5200,62.3350,765626.0,0.1850,0.060042,NaN
2022-02-01 09:30:15,986,62.3500,62.2501,62.3700,62.1700,225789.0,0.2000,0.070039,NaN
2022-02-01 09:30:30,987,62.2600,62.0400,62.2677,62.0300,177411.0,0.2377,0.082015,NaN
2022-02-01 09:30:45,988,62.0400,61.9089,62.0495,61.8600,421885.0,0.1895,0.089692,NaN
2022-02-01 09:31:00,989,61.8900,61.8700,61.9100,61.7100,232588.0,0.2000,0.097571,NaN
...,...,...,...,...,...,...,...,...,...
2024-02-01 15:57:00,1632564,54.3500,54.3650,54.3700,54.3400,44530.0,0.0300,0.040529,NaN
2024-02-01 15:57:15,1632565,54.3623,54.3640,54.3900,54.3600,59581.0,0.0300,0.039777,NaN
2024-02-01 15:57:30,1632566,54.3600,54.3900,54.3900,54.3599,33387.0,0.0301,0.039086,NaN


In [16]:
df_test = df[df.index<pd.to_datetime('2022-03-01')]
df_test

,Unnamed: 0,Open,Close,High,Low,Volume,tr,atr,entries
Time,,,,,,,,,
2022-02-01 09:30:00,985,62.4200,62.3500,62.5200,62.335,765626.0,0.1850,0.060042,NaN
2022-02-01 09:30:15,986,62.3500,62.2501,62.3700,62.170,225789.0,0.2000,0.070039,NaN
2022-02-01 09:30:30,987,62.2600,62.0400,62.2677,62.030,177411.0,0.2377,0.082015,NaN
2022-02-01 09:30:45,988,62.0400,61.9089,62.0495,61.860,421885.0,0.1895,0.089692,NaN
2022-02-01 09:31:00,989,61.8900,61.8700,61.9100,61.710,232588.0,0.2000,0.097571,NaN
...,...,...,...,...,...,...,...,...,...
2022-02-28 15:57:00,61572,52.2100,52.2600,52.2696,52.160,188072.0,0.1096,0.125000,NaN
2022-02-28 15:57:15,61573,52.2510,52.2700,52.2900,52.250,133174.0,0.0400,0.118929,NaN
2022-02-28 15:57:30,61574,52.2701,52.2850,52.3700,52.230,108691.0,0.1400,0.120434,NaN


In [32]:
entries = [False]
exits = []
countCandleV,startCount = 0,0
for i in range(1,len(df_test)):
    dff=df.iloc[:i,:]
    if dff.iloc[-1].name.time() == time(9, 30, 0):
        countCandleV,startCount = 0,0

    v,countCandleV,startCount = V(dff.iloc[-1]['Close'],dff.iloc[-1]['Open'],dff.iloc[-1]['tr'],dff.iloc[-1]['atr'],startCount,countCandleV)

    entries.append(v)
    
    

In [31]:
len(entries)

29506

In [33]:
df_test['ent'] = entries
df_test['ext'] = False

In [34]:
entries = df_test['ent']
exits = df_test['ext']

In [36]:
exits.value_counts()

ext
False    29507
Name: count, dtype: int64

In [ ]:
# returns_df = pd.DataFrame(columns = ['return'])
# for i in np.arange(0.1,2.5, 0.1):
#     for j in np.arange(0.1, 1, 0.1):
#         trading_time_idx = df_test.Close.between_time('9:30', '16:00', inclusive='left').index
#         entries[~entries.index.isin(trading_time_idx)] = False
#         exits[~exits.index.isin(trading_time_idx)] = False
#         exits.loc[datetime.time(16, 0)] = True
#         pf = vbt.Portfolio.from_signals(df.Close, entries, exits, init_cash=100000, freq='15s', size_granularity=1, sl_stop=i/100, tp_stop=j/100)
#         returns_df.loc[f'tp:{round(i,1)}, sl:{round(j,1)}'] = [pf.total_return()*100]